#### NLP 答疑课 (08.04.2019)
outline:
+ edit distance 的 parsed_solution 程序
+ word2vec 以及在gensim里的参数设定

课程内容recap:
+ 动态规划问题具有哪些特点？如何解析solution？
+ 自然语言处理中为什么需要 word embedding ？
+ word2vec 的基本原理是什么？

参考资料 (optional)：
+ solutions for the traveling-salesman-problem (formally, a np-complete problem):
https://github.com/rohanp/travelingSalesman;
https://www.csd.uoc.gr/~hy583/papers/ch11.pdf
+ word2vec 的原论文（Mikolov et al. 2013）:
https://arxiv.org/abs/1301.3781
+ skip-gram word2vec tutorial:
http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
+ improvements on skip-gram (hierarchical softmax & negative sampling):
https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf
+ According to the authors, hierarchical softmax works better for infrequent words while negative sampling works better for frequent words and better with low dimensional vectors.


#### Edit distance

In [1]:
solution = {}

In [2]:
from functools import lru_cache

In [3]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL{}'.format(tail_s1)),
        # delete tail_s1 for string 1; edit_distance + 1 for each deletion
        (edit_distance(string1, string2[:-1]) + 1, 'ADD{}'.format(tail_s2))
        # add tail_s2 to string 1; edit_distance + 1 for each addition
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
        # no edits
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 
                        'SUB {} => {}'.format(tail_s1, tail_s2))
        # substitute tail_s2 for tail_s1; edit_distance + 1 for each substitution
        
    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    # find the minimal edit distance among all possible operations
    
    solution[(string1, string2)] = operation # store solution to dictionary
    
    return min_distance

In [4]:
edit_distance('ABCD', 'ABCDE')

1

In [5]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADDB',
 ('A', 'ABC'): 'ADDC',
 ('A', 'ABCD'): 'ADDD',
 ('A', 'ABCDE'): 'ADDE',
 ('AB', 'A'): 'DELB',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADDC',
 ('AB', 'ABCD'): 'ADDD',
 ('AB', 'ABCDE'): 'ADDE',
 ('ABC', 'A'): 'DELC',
 ('ABC', 'AB'): 'DELC',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCD'): 'ADDD',
 ('ABC', 'ABCDE'): 'ADDE',
 ('ABCD', 'A'): 'DELD',
 ('ABCD', 'AB'): 'DELD',
 ('ABCD', 'ABC'): 'DELD',
 ('ABCD', 'ABCD'): '',
 ('ABCD', 'ABCDE'): 'ADDE'}

In [6]:
edit_distance('BEIJING','NANJING')

3

In [7]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADDB',
 ('A', 'ABC'): 'ADDC',
 ('A', 'ABCD'): 'ADDD',
 ('A', 'ABCDE'): 'ADDE',
 ('AB', 'A'): 'DELB',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADDC',
 ('AB', 'ABCD'): 'ADDD',
 ('AB', 'ABCDE'): 'ADDE',
 ('ABC', 'A'): 'DELC',
 ('ABC', 'AB'): 'DELC',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCD'): 'ADDD',
 ('ABC', 'ABCDE'): 'ADDE',
 ('ABCD', 'A'): 'DELD',
 ('ABCD', 'AB'): 'DELD',
 ('ABCD', 'ABC'): 'DELD',
 ('ABCD', 'ABCD'): '',
 ('ABCD', 'ABCDE'): 'ADDE',
 ('B', 'N'): 'SUB B => N',
 ('B', 'NA'): 'ADDA',
 ('B', 'NAN'): 'ADDN',
 ('B', 'NANJ'): 'ADDJ',
 ('B', 'NANJI'): 'ADDI',
 ('B', 'NANJIN'): 'ADDN',
 ('B', 'NANJING'): 'ADDG',
 ('BE', 'N'): 'DELE',
 ('BE', 'NA'): 'SUB E => A',
 ('BE', 'NAN'): 'ADDN',
 ('BE', 'NANJ'): 'ADDJ',
 ('BE', 'NANJI'): 'ADDI',
 ('BE', 'NANJIN'): 'ADDN',
 ('BE', 'NANJING'): 'ADDG',
 ('BEI', 'N'): 'DELI',
 ('BEI', 'NA'): 'DELI',
 ('BEI', 'NAN'): 'SUB I => N',
 ('BEI', 'NANJ'): 'ADDJ',
 ('BEI', 'NANJI'): '',
 ('BEI', 'NANJIN'): 'ADDN',
 ('BEI', 'NANJIN

In [10]:
# to parse the solutions, we need to define the exit condition ("查表"过程的终止条件)
# in this case, the exit condition is
# -- if the two strings are exactly the same, we stop editing

def stop_edit(string1, string2): return string1 == string2


# now we look for solutions in the "solution" dictionary

def parse_solution(string1, string2, solution_dic):
    
    parsed_solutions = [] 
    
    while not stop_edit(string1, string2):
        operation = solution_dic[(string1, string2)] # "查表" 过程
        
        if 'SUB' in operation:
            string1, string2 = string1[:-1], string2[:-1] 
            # if substitution, both forward and compare
        elif operation == '':
            string1, string2 = string1[:-1], string2[:-1]
            # if no edits, both forward and compare
        elif 'DEL' in operation:
            string1, string2 = string1[:-1], string2 
            # delete tail of string1 and then compare
        elif 'ADD' in operation:
            string1, string2 = string1, string2[:-1] 
            # delete tail of string2 and then compare
        
        parsed_solutions.append(operation)
    
    return parsed_solutions


In [11]:
parse_solution('BEIJING','NANJING', solution)

['', '', '', '', 'SUB I => N', 'SUB E => A', 'SUB B => N']

#### implementing word2vec using gensim

In [1]:
from gensim.models.word2vec import LineSentence
from gensim.models import Word2Vec

In [2]:
# Initialize and train a simple Word2Vec model

sentences = [["cat", "say", "meow"], ["dog", "say", "woof"]]
model = Word2Vec(sentences, min_count=1)

In [3]:
model.most_similar('cat') # check out the most similar words for "cat"

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('say', 0.15734419226646423),
 ('dog', -0.02822110615670681),
 ('woof', -0.054968975484371185),
 ('meow', -0.09244367480278015)]

In [4]:
model.wv['dog'] # check out the word vector for 'dog'

# note 
# -- the default size of the world vector is 100

array([-4.6197851e-03, -3.7394990e-03, -1.9653954e-03,  4.9299968e-04,
       -1.7356514e-03,  1.3853174e-03, -1.3880365e-03,  3.7120937e-03,
        4.0574811e-04, -6.7226618e-04,  4.1654790e-03, -1.7620082e-03,
       -2.2858770e-03, -1.2873072e-03,  1.3976595e-03,  3.2130738e-03,
        4.0538125e-03, -2.0839639e-04,  2.0860375e-03,  1.9286850e-03,
        4.2720833e-03,  5.3239003e-04,  1.0568922e-03, -4.6414379e-03,
       -1.1774941e-03,  3.7658615e-03,  4.1018156e-04,  7.8001170e-04,
        4.9522161e-03, -9.2942256e-04,  4.3550124e-03,  4.3893387e-03,
        8.4427436e-04, -3.0914419e-03,  4.6283836e-03,  1.2586201e-03,
       -4.6638153e-03,  2.6103409e-03, -9.8581535e-05,  4.6953690e-05,
        2.3059270e-03,  4.2193383e-03, -3.2957387e-03,  2.6589034e-03,
        2.5297115e-03, -4.0766471e-03, -2.6186807e-03,  1.2288799e-03,
        1.0929701e-03,  4.7582344e-04,  4.1757482e-03, -1.0421752e-03,
       -2.5041485e-03, -3.1426621e-03,  1.3271449e-03,  3.8764635e-03,
      

In [5]:
# Let's try word2vec using a larger corpus (the chinese news corpus in LineSentence format)

line_setences_path = '/Users/xinweixu/Dropbox/learn/Comp_Prog/nlp/data/sentences-cut.txt'
sentences = LineSentence(line_setences_path)

In [6]:
%time # check out the cell execution time
news_model_1 = Word2Vec(sentences, min_count=5, size = 50)
# notes:
# -- ignore words appearing less than 5 times in the corpus
# -- set the word vector size to be 50
# -- size: the dimensionality of the vector, or the size of the neural net layers, 
#    the default is 100, and larger size would require more training data!


# See documentation here: 
# https://radimrehurek.com/gensim/models/word2vec.html

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.91 µs


In [7]:
news_model_1.window # the default window size is 5 (we used a default setting of CBOW)

# -- window: how many words before and after a given word would be 
#    included as context words of the given word.
#    According to Mikolov et al.'s notes, the recommended value is 10 for skip-gram and 5 for CBOW

5

In [8]:
news_model_1.wv['霍金'] # check out the word vector for '霍金'

array([-0.6583145 ,  0.23051035,  0.35316232, -0.12076233, -0.23267202,
       -0.3545135 ,  0.37806377, -0.84230447,  0.19441096, -0.3814887 ,
        0.12050633, -0.23351657, -0.0562492 ,  0.32234514,  0.36346287,
       -0.13334295, -0.06037512, -0.23620208, -0.12063441,  0.30764708,
        0.0938347 ,  0.03143591, -0.48876804,  0.35775042,  0.12461712,
       -0.16470769, -0.5739129 ,  0.21065676, -0.00166393,  0.639909  ,
        0.37221038, -0.48421142, -0.01305088,  0.32505998, -0.3972132 ,
        0.09074134, -0.1852542 , -0.25409117,  0.27777195,  0.39134544,
       -0.26426288,  0.2804928 , -0.33939067,  0.03918035, -0.20144188,
       -0.14565597, -0.38240653,  0.18649364, -0.09471973, -0.16158222],
      dtype=float32)

In [9]:
news_model_1.most_similar('霍金', topn=20) # check out the top 20 most similar words to '霍金'

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('外星人', 0.7982823252677917),
 ('发表文章', 0.7825572490692139),
 ('这番话', 0.737501323223114),
 ('两篇', 0.7329206466674805),
 ('凯利', 0.7296558022499084),
 ('佳佳', 0.7218890190124512),
 ('研究者', 0.720791757106781),
 ('倪光南', 0.7181127071380615),
 ('署名文章', 0.7146850228309631),
 ('半月刊', 0.7076667547225952),
 ('谢泽雄', 0.6993716955184937),
 ('心理学家', 0.6986111402511597),
 ('如是', 0.6861940026283264),
 ('引用', 0.684630274772644),
 ('乔纳森', 0.6833487153053284),
 ('李启威', 0.6798511147499084),
 ('旁人', 0.679509699344635),
 ('这篇', 0.679101824760437),
 ('华尔特', 0.6787354946136475),
 ('类人猿', 0.678704023361206)]

How would the results differ if we use a different model set-up?

In [10]:
%time
news_model_2 = Word2Vec(sentences, min_count=5, size = 150, window = 10, sg = 1, workers = 8)

# notes:
# -- this time we set the word vector size to be 150, context window to be 10 words,
#    and use the skip-gram algorithm (which takes longer time!!!)
# -- sg: training algorithm -- 1 for skip-gram; otherwise CBOW.
# -- workers: speed up computing time using multiple cores 
#   (capped by the number of cores on your machine!!!)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.15 µs


In [12]:
news_model_2.wv['霍金']

array([ 0.09543171,  0.36387163,  0.53725225,  0.500516  , -0.16229299,
       -0.11686423,  0.47798058, -0.2476761 , -0.49148753, -0.32124692,
       -0.46384513, -0.04520119, -0.04683913,  0.06340239, -0.17569564,
       -0.09566884,  0.10246837, -0.08239821, -0.13604444,  0.12139518,
        0.17819159, -0.04748364, -0.3483194 ,  0.00635009, -0.21259819,
        0.01808786, -0.1372791 , -0.13192311, -0.3287806 ,  0.20427483,
        0.05614374,  0.07496247,  0.26630557, -0.14758518,  0.24570504,
       -0.18302245, -0.04394631, -0.30380732, -0.11139403,  0.26414317,
        0.05811457,  0.24270004, -0.09761038,  0.38712123, -0.08975498,
        0.55450153, -0.06434401,  0.24952   ,  0.1934675 , -0.03136092,
       -0.01793445,  0.06577613, -0.32730475, -0.02902058,  0.749861  ,
        0.06751074,  0.25108746,  0.07870406, -0.05638078,  0.26128837,
        0.22157706,  0.40108484, -0.1727779 , -0.38061875,  0.80699366,
        0.31915122, -0.00987758,  0.15248184,  0.49250683, -0.11

In [13]:
news_model_2.most_similar('霍金', topn=20)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('霍金的', 0.7316663861274719),
 ('双周刊', 0.7126336097717285),
 ('史密森', 0.7081307172775269),
 ('NASA', 0.6939337849617004),
 ('学术刊物', 0.6935392022132874),
 ('哈勃', 0.6920583248138428),
 ('美国航空航天局', 0.6904950141906738),
 ('奥斯', 0.6795668005943298),
 ('韦尔塔', 0.6710573434829712),
 ('美国国家航空航天局', 0.6707107424736023),
 ('伯杰', 0.6678393483161926),
 ('赫芬顿', 0.664324164390564),
 ('Nature', 0.6638203859329224),
 ('华尔特', 0.6630619168281555),
 ('博伊尔', 0.6625248789787292),
 ('朱清时', 0.6623486876487732),
 ('外星人', 0.6581864953041077),
 ('美国加利福尼亚大学', 0.6579184532165527),
 ('博尔顿', 0.6562376618385315),
 ('Science', 0.6560503244400024)]

Now let's save the word vectors to a local directory for future use!

In [11]:
# first, we can save the model (you can change the filename to a desirable local directory)
news_model_1.save('news_model_1')
news_model_2.save('news_model_2')

In [26]:
# to load a model, use the following:
from gensim.test.utils import datapath

news_model_1 = Word2Vec.load('news_model_1')
news_model_2 = Word2Vec.load('news_model_2')

In [27]:
# second, we can also save the word vectors for future query
word_vectors_1 = news_model_1.wv
word_vectors_1.save('news_model_1_word_vectors.kv')

word_vectors_2 = news_model_2.wv
word_vectors_2.save('news_model_2_word_vectors.kv')

In [31]:
# to load word vectors from an existing file
from gensim.models import KeyedVectors

word_vectors_1 = KeyedVectors.load('news_model_1_word_vectors.kv')
word_vectors_2 = KeyedVectors.load('news_model_2_word_vectors.kv')

See more info on saving word2vec models:
https://radimrehurek.com/gensim/models/keyedvectors.html